In [1]:
def convert_time(df, time_column):
    # Check if the time data represents an interval
    if " - " in df[time_column].iloc[0]:
        # Split the time interval into start and end times
        df["Start Time"] = df[time_column].str.split(
            " - ", expand=True
        )[0]
        #drop initial time column
        df.drop(time_column,inplace=True,axis=1)
        # create a new column with only the start time of Time column 
        df["Start Time"] = pd.to_datetime(df["Start Time"], format="%d.%m.%Y %H:%M")
        #assign to the column with the same initial name (time_column)
        df[time_column]=df['Start Time']
        #drop Start Time column 
        df.drop('Start Time',inplace=True,axis=1)
    else:
        # Convert the time data to datetime
        df[time_column] = pd.to_datetime(df[time_column], format="%d.%m.%Y %H:%M")

    return df

# Apply the function to your DataFrame


In [2]:
import pandas as pd 

df = pd.read_csv("Total Load - Day Ahead _ Actual_202301010000-202401010000 (3).csv")


In [3]:
df.head()

,Time (CET/CEST),Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL)
0,01.01.2023 00:00 - 01.01.2023 00:15,11330,9912
1,01.01.2023 00:15 - 01.01.2023 00:30,11251,9813
2,01.01.2023 00:30 - 01.01.2023 00:45,11154,9877
3,01.01.2023 00:45 - 01.01.2023 01:00,11045,9861
4,01.01.2023 01:00 - 01.01.2023 01:15,10929,9847


In [4]:
time_column='Time (CET/CEST)'

In [5]:
df_read = convert_time(df, time_column)

In [13]:
df[time_column].str.split(
            " - ", expand=True
        )[0]

0        01.01.2023 00:00
1        01.01.2023 00:15
2        01.01.2023 00:30
3        01.01.2023 00:45
4        01.01.2023 01:00
               ...       
35039    31.12.2023 22:45
35040    31.12.2023 23:00
35041    31.12.2023 23:15
35042    31.12.2023 23:30
35043    31.12.2023 23:45
Name: 0, Length: 35044, dtype: object

In [6]:
df_read

,Time (CET/CEST),Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL),Start Time
0,01.01.2023 00:00 - 01.01.2023 00:15,11330,9912,2023-01-01 00:00:00
1,01.01.2023 00:15 - 01.01.2023 00:30,11251,9813,2023-01-01 00:15:00
2,01.01.2023 00:30 - 01.01.2023 00:45,11154,9877,2023-01-01 00:30:00
3,01.01.2023 00:45 - 01.01.2023 01:00,11045,9861,2023-01-01 00:45:00
4,01.01.2023 01:00 - 01.01.2023 01:15,10929,9847,2023-01-01 01:00:00
...,...,...,...,...
35039,31.12.2023 22:45 - 31.12.2023 23:00,-,-,2023-12-31 22:45:00
35040,31.12.2023 23:00 - 31.12.2023 23:15,-,-,2023-12-31 23:00:00
35041,31.12.2023 23:15 - 31.12.2023 23:30,-,-,2023-12-31 23:15:00
35042,31.12.2023 23:30 - 31.12.2023 23:45,-,-,2023-12-31 23:30:00


In [9]:
df_read.drop('Time (CET/CEST)',inplace=True,axis=1)

In [11]:
df_read[time_column]=df_read['Start Time']

In [13]:
df_read.drop('Start Time',inplace=True,axis=1)

In [14]:
df_read

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL),Time (CET/CEST)
0,11330,9912,2023-01-01 00:00:00
1,11251,9813,2023-01-01 00:15:00
2,11154,9877,2023-01-01 00:30:00
3,11045,9861,2023-01-01 00:45:00
4,10929,9847,2023-01-01 01:00:00
...,...,...,...
35039,-,-,2023-12-31 22:45:00
35040,-,-,2023-12-31 23:00:00
35041,-,-,2023-12-31 23:15:00
35042,-,-,2023-12-31 23:30:00


In [16]:
df_read[time_column]=pd.to_datetime(df_read[time_column], format="%d.%m.%Y %H:%M")

In [17]:
df_read

,Day-ahead Total Load Forecast [MW] - Netherlands (NL),Actual Total Load [MW] - Netherlands (NL),Time (CET/CEST)
0,11330,9912,2023-01-01 00:00:00
1,11251,9813,2023-01-01 00:15:00
2,11154,9877,2023-01-01 00:30:00
3,11045,9861,2023-01-01 00:45:00
4,10929,9847,2023-01-01 01:00:00
...,...,...,...
35039,-,-,2023-12-31 22:45:00
35040,-,-,2023-12-31 23:00:00
35041,-,-,2023-12-31 23:15:00
35042,-,-,2023-12-31 23:30:00
